(c) 2023 Manuel Razo. This work is licensed under a [Creative Commons
Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/).
All code contained herein is licensed under an [MIT
license](https://opensource.org/licenses/MIT).

In [1]:
# Load project package
@load_pkg BayesFitUtils

# Import project package
import BayesFitUtils

# Import package to handle DataFrames
import DataFrames as DF
import CSV
import Tables

# Import package to load chains
import JLD2

# Import package to handle MCMC chain objects
import MCMCChains

# Import basic statistical functions
import StatsBase
import Distributions

# Import library to list files
import Glob

# Load CairoMakie for plotting
using CairoMakie
import ColorSchemes
import Makie
# Activate backend
CairoMakie.activate!()

# Set PBoC Plotting style
BayesFitUtils.viz.pboc_makie!()

# Fitness inference exploratory data analysis for Kinsler et al., 2020

In this notebook, we will explore the output of the inferences done on the
Kinsler et al., 2020 datasets.

Let us begin by loading the inferences done exclusively on the neutral linages
to infer the prior values for the full joint inference.

In [4]:
# Define directory
out_dir = "$(git_root())/code/processing/mcmc_kinsler_fitness/" *
          "output/popmean_fitness"

# List files
files = Glob.glob("$(out_dir)/kinsler*jld2"[2:end], "/")

println("# of files = $(length(files))")

# of files = 51


Now, let's loop through the files, load the chains, and fit parametric
distributions to the relevant parameters to determine the value of the priors.
To better organize all of this information, we will store the values on a tidy
dataframe.

In [7]:
split(files[1], "/")[end]

"kinsler_0.2MKClenv_R1rep_falsermT0_1000steps_04walkers.jld2"

In [11]:
# Initialize dataframe to save distribution parameters
df_popmean = DF.DataFrame()

# Loop through files
for f in files
    # Split file name
    f_split = split(split(f, "/")[end], "_")

    # Extract file metadata
    env = replace(f_split[2], "env" => "")
    rep = replace(f_split[3], "rep" => "")

    # Load chain into memory
    chn = JLD2.load(f)["chain"]

    # Select variables for population mean fitness and associated variance
    var_name = MCMCChains.namesingroup.(Ref(chn), [:s̲ₜ, :σ̲ₜ])

    # Fit normal distributions to population mean fitness
    pop_mean = Distributions.fit.(
        Ref(Distributions.Normal), [vec(chn[x]) for x in var_name[1]]
    )

    # Fit lognormal distributions to associated error
    pop_std = Distributions.fit.(
        Ref(Distributions.LogNormal), [vec(chn[x]) for x in var_name[2]]
    )

    # Extract distribution parameters into matrix
    df_param = DF.DataFrame(
        hcat(
            first.(Distributions.params.(pop_mean)),
            last.(Distributions.params.(pop_mean)),
            first.(Distributions.params.(pop_std)),
            last.(Distributions.params.(pop_std))
        ),
        ["s_mean", "s_std", "σ_mean", "σ_std"]
    )

    # Add columns for time and metadata
    DF.insertcols!(
        df_param,
        :env .=> env,
        :rep .=> rep,
        :time => axes(df_param, 1)
    )

    # Append to dataframe
    DF.append!(df_popmean, df_param)
end # for

# Write dataframe into CSV file
CSV.write("$(out_dir)/popmean_fitness_priors.csv", df_popmean)

first(df_popmean, 8)

Row,s_mean,s_std,σ_mean,σ_std,env,rep,time
,Float64,Float64,Float64,Float64,String,String,Int64
1,0.451544,0.0448838,-1.98867,0.336591,0.2MKCl,R1,1
2,0.400424,0.0578217,-1.70192,0.2836,0.2MKCl,R1,2
3,0.429273,0.072205,-1.53064,0.307813,0.2MKCl,R1,3
4,0.366196,0.0835919,-1.43045,0.304358,0.2MKCl,R1,4
5,0.390209,0.0711818,-1.54629,0.286934,0.2MNaCl,R1,1
6,0.36445,0.0548963,-1.75539,0.304978,0.2MNaCl,R1,2
7,0.534052,0.0742311,-1.48813,0.308161,0.2MNaCl,R1,3
8,0.32696,0.0787879,-1.47863,0.333311,0.2MNaCl,R1,4


We use these values to set the priors for the global joint inference.
Specifically, we use the parameters for the population mean fitness values
(`s_mean`, and `s_std`) to set the priors for the population mean fitness when
performing the joint inference. The same for the parameters on the standard
deviation of the likelihood function for the neutral lineages. Furthermore,
we use these latter parameters also as the priors on the standard deviation of
the likelihood function for the mutant lineages.

Let's repeat an equivalent analysis for the barcode frequencies. First, let's
list the files.

In [12]:
# Define directory
out_dir = "$(git_root())/code/processing/mcmc_kinsler_fitness/" *
          "output/bc_freq"

# List files
files = Glob.glob("$(out_dir)/kinsler*jld2"[2:end], "/")

println("# of files = $(length(files))")

# of files = 51


Next, let's fit parametric distributions to the Poisson parameter values.

In [18]:
# Initialize dataframe to save distribution parameters
df_freq = DF.DataFrame()

# Loop through files
for f in files
    # Split file name
    f_split = split(split(f, "/")[end], "_")

    # Extract file metadata
    env = replace(f_split[2], "env" => "")
    rep = replace(f_split[3], "rep" => "")

    # Load chain into memory
    chn = JLD2.load(f)["chain"]

    # Select variables for population mean fitness and associated variance
    var_name = MCMCChains.namesingroup(chn, :Λ̲̲)

    # Fit normal distributions to population mean fitness
    bc_freq = Distributions.fit.(
        Ref(Distributions.LogNormal), [vec(chn[x]) for x in var_name]
    )

    # Extract distribution parameters into matrix
    df_param = DF.DataFrame(
        hcat(
            first.(Distributions.params.(bc_freq)),
            last.(Distributions.params.(bc_freq)),
            StatsBase.mean.(bc_freq),
            StatsBase.std.(bc_freq),
        ),
        ["λ_mean", "λ_std", "bc_mean", "bc_std"]
    )

    # Add columns for time and metadata
    DF.insertcols!(
        df_param,
        :env .=> env,
        :rep .=> rep,
    )

    # Append to dataframe
    DF.append!(df_freq, df_param)
end # for

# Write dataframe into CSV file
CSV.write("$(out_dir)/bc_freq_priors.csv", df_freq)

first(df_freq, 8)

Row,λ_mean,λ_std,bc_mean,bc_std,env,rep
,Float64,Float64,Float64,Float64,String,String
1,8.73987,0.0110089,6247.43,68.7797,0.2MKCl,R1
2,7.75991,0.0219595,2345.25,51.5066,0.2MKCl,R1
3,7.37042,0.0254693,1588.81,40.4725,0.2MKCl,R1
4,4.59857,0.107446,99.9174,10.7668,0.2MKCl,R1
5,4.92564,0.0779711,138.196,10.7917,0.2MKCl,R1
6,8.66376,0.0130694,5789.76,75.6719,0.2MKCl,R1
7,7.74126,0.02048,2301.86,47.1469,0.2MKCl,R1
8,7.34235,0.0205986,1544.66,31.8211,0.2MKCl,R1


## Loading data and inferences

Let's begin by loading the raw data.

In [ ]:
data = CSV.read(
    "$(git_root())/data/kinsler_2020/tidy_counts_no_anc.csv", DF.DataFrame
)

first(data, 3)

Next, let's list the `.jld2` files containing the MCMC chains.

In [ ]:
# List files
chn_files = Glob.glob(
    "$(git_root())/code/processing/mcmc_kinsler_fitness/output/"[2:end] *
    "kinsler*jld2",
    "/"
)

println("# files = $(length(chn_files))")

Let us begin exploring one of these files to later generalize the analysis
pipeline. Each `.jld2` file contains two objects:
- `chain`: The `MCMCChains.Chain` object with all the samples.
- `group`: An array listing the identity of each of the mutants in the order
  they follow in the chain.

Let's load the chain and rename the variables to the barcode names.

In [ ]:
# Load chain and group object form .jld2 file
chn, group = values(JLD2.load(chn_files[1]))

# Search for variable names
s_names = MCMCChains.namesingroup(chn, "s̲⁽ᵐ⁾")
# Search for variable names
σ_names = MCMCChains.namesingroup(chn, "σ̲⁽ᵐ⁾")

# Rename variables
chn = MCMCChains.replacenames(
    chn,
    Dict(
        zip(
            [s_names; σ_names],
            [["s[$(x)]" for x in group]; ["σ[$(x)]" for x in group]]
        )
    )
)

## Exploring inferred distributions symmetry

The simplest way to summarize the results is to fit a parametric distribution to
the MCMC samples. Ideally, we can use a simple symmetric distribution such as a
Gaussian. A necessary requirement for this is that the skewness of the MCMC
chains should be very close to zero as

Let us compute the skewness and kurtosis for all the chains.

In [ ]:
# Define variable fitness variable names
var_names = MCMCChains.namesingroup(chn, "s")

# Convert chain to tidy dataframe
df = DF.stack(DF.DataFrame(chn[var_names]), DF.Not([:iteration, :chain]))

# Compute skewness and kurtosis
df_summary = DF.combine(
    DF.groupby(df, :variable),
    :value => StatsBase.median,
    :value => StatsBase.mean,
    :value => StatsBase.std,
    :value => StatsBase.var,
    :value => StatsBase.skewness,
    :value => StatsBase.kurtosis
)

first(df_summary, 5)

Let's plot the distribution of these values.

In [ ]:
# Initialize figure
fig = Figure(resolution=(300 * 2, 300 * 2))

# Add axis
axes = [
    Axis(
        fig[i, j],
        ylabel="ECDF"
    ) for i = 1:2 for j = 1:2
]

# List quantities to plot
qs = ["mean", "var", "skewness", "kurtosis"]

# Loop through quantities
for (i, q) in enumerate(qs)
    # Plot quantity
    ecdfplot!(axes[i], df_summary[:, "value_"*q])
    # Add title
    axes[i].xlabel = "fitness $(q)"
end # for

fig

Focusing in particular on the skewness⸺which we expect to be zero for a
symmetric distribution⸺the vast majority of the density concentrates around
zero, but there are some extreme values. Let's look at a couple of distributions
with extreme skewness values.

In [ ]:
# Identify minimum and maximum value of skewness
min_skew, max_skew = [
    df_summary[argmin(df_summary.value_skewness), :variable]
    df_summary[argmax(df_summary.value_skewness), :variable]
]

# Initialize plot
fig = Figure(resolution=(300 * 2, 300))

# Add axis
axes = [
    Axis(
        fig[1, i],
        xlabel="fitness",
        ylabel="density"
    ) for i = 1:2
]

# Define plot titles 
titles = ["min skewness", "max skewness"]

# loop through barcodes
for (i, bc) in enumerate([min_skew, max_skew])
    # plot density for fitness value
    density!(axes[i], df[df.variable.==bc, :value])
    # Add title
    axes[i].title = titles[i]
end # for

fig

We can see that even the distributions with extreme skewness values are fairly
symmetric. The high skewness must come from the few outliers in the samples.
Let's repeat the same plot, this time only including the 95% highest density
percentile.

In [ ]:
# Identify minimum and maximum value of skewness
min_skew, max_skew = [
    df_summary[argmin(df_summary.value_skewness), :variable]
    df_summary[argmax(df_summary.value_skewness), :variable]
]

# Initialize plot
fig = Figure(resolution=(300 * 2, 300))

# Add axis
axes = [
    Axis(
        fig[1, i],
        xlabel="fitness",
        ylabel="density"
    ) for i = 1:2
]

# Define plot titles 
titles = ["min skewness", "max skewness"]

# loop through barcodes
for (i, bc) in enumerate([min_skew, max_skew])
    # Extract values
    skew = df[df.variable.==bc, :value]
    # Keep only selected percentile
    skew = skew[
        (skew.≥StatsBase.percentile(skew, 2.5)).&(skew.≤StatsBase.percentile(skew, 97.5))
    ]
    # plot density for fitness value
    density!(axes[i], skew)
    # Add title
    axes[i].title = titles[i]
end # for

fig

Removing the extreme values does return a reasonably symmetric distribution.
Let's compute the summary statistics only using the 95% percentile.

In [ ]:
# Group data by barcode
df_group = DF.groupby(df, :variable)

# Initialize dataframe to save summaries
df_filt = DF.DataFrame()

# Loop through groups
for g in df_group
    # Filter data
    DF.append!(
        df_filt,
        g[(g.value.≥StatsBase.percentile(g.value, 5)).&(g.value.≤StatsBase.percentile(g.value, 95)),
            :]
    )
end # group

# Compute skewness and kurtosis
df_filt_summary = DF.combine(
    DF.groupby(df_filt, :variable),
    :value => StatsBase.median,
    :value => StatsBase.mean,
    :value => StatsBase.std,
    :value => StatsBase.var,
    :value => StatsBase.skewness,
    :value => StatsBase.kurtosis
)

first(df_filt_summary, 5)

As before, Let's plot the distribution of these summary statistics.

In [ ]:
# Initialize figure
fig = Figure(resolution=(300 * 2, 300 * 2))

# Add axis
axes = [
    Axis(
        fig[i, j],
        ylabel="ECDF"
    ) for i = 1:2 for j = 1:2
]

# List quantities to plot
qs = ["mean", "var", "skewness", "kurtosis"]

# Loop through quantities
for (i, q) in enumerate(qs)
    # Plot quantity
    ecdfplot!(axes[i], df_filt_summary[:, "value_"*q])
    # Add title
    axes[i].xlabel = "fitness $(q)"
end # for

fig

Focusing again on the skewness, we still find some values that deviate from the
expected zero-value for symmetric distributions. Let's plot the probability
density for the fitness value of these extreme values.

In [ ]:
# Identify minimum and maximum value of skewness
min_skew, max_skew = [
    df_filt_summary[argmin(df_filt_summary.value_skewness), :variable]
    df_filt_summary[argmax(df_filt_summary.value_skewness), :variable]
]

# Initialize plot
fig = Figure(resolution=(300 * 2, 300))

# Add axis
axes = [
    Axis(
        fig[1, i],
        xlabel="fitness",
        ylabel="density"
    ) for i = 1:2
]

# Define plot titles 
titles = ["min skewness", "max skewness"]

# loop through barcodes
for (i, bc) in enumerate([min_skew, max_skew])
    # plot density for fitness value
    density!(axes[i], df_filt[df_filt.variable.==bc, :value])
    # Add title
    axes[i].title = titles[i]
end # for

fig

These are indeed not symmetric. But notice the range of the density plots; these
extreme skewness values com from poorly identified fitness values. The
distribution of variance values above reveals a set of identified and poorly
identified fitness values. Let's plot this distribution to emphasize the point.

In [ ]:
# Initialize figure
fig = Figure(resolution=(350, 350))

# Add axis
ax = Axis(
    fig[1, 1],
    xlabel="fitness variance",
    ylabel="density",
)

# Plot density
density!(ax, df_filt_summary[:, :value_var])

# Set x-axis range
xlims!(ax, [-0.05, 0.4])

fig

Most of the barcode inferences have a variance $\leq$ to 0.1. Let us filter out
everything above this value and look at the skewness distribution.

In [ ]:
# Initialize figure
fig = Figure(resolution=(350, 350))

# Add axis
ax = Axis(
    fig[1, 1],
    xlabel="fitness skewness",
    ylabel="density",
    title="var ≤ 0.1"
)

# Plot density
ecdfplot!(
    ax, df_filt_summary[df_filt_summary.value_var.≤0.1, :value_skewness]
)

fig

As expected, removing the high-variance estimates reduces the range of skewness
values. Let's look at the extreme distributions for these further-filtered
barcodes.

In [ ]:
# Filter barcodes by variance
df_filt_summary_var = df_filt_summary[df_filt_summary.value_var.≤0.1, :]

# Identify minimum and maximum value of skewness
min_skew, max_skew = [
    df_filt_summary_var[argmin(df_filt_summary_var.value_skewness), :variable]
    df_filt_summary_var[argmax(df_filt_summary_var.value_skewness), :variable]
]

# Initialize plot
fig = Figure(resolution=(300 * 2, 300))

# Add axis
axes = [
    Axis(
        fig[1, i],
        xlabel="fitness",
        ylabel="density"
    ) for i = 1:2
]

# Define plot titles 
titles = ["min skewness", "max skewness"]

# loop through barcodes
for (i, bc) in enumerate([min_skew, max_skew])
    # plot density for fitness value
    density!(axes[i], df_filt[df_filt.variable.==bc, :value])
    # Add title
    axes[i].title = titles[i]
end # for

fig

The distributions are indeed fairly symmetric when filtering out the
high-variance inferences.

## Systematic fitting of Gaussian distributions

Where does this analysis leave us? What we can safely conclude is that for the
vast majority of the barcodes, fitting a parametric Gaussian curve to the MCMC
chains is a good-enough approximation. This does not generalize to all cases,
but we might consider discarding the cases that fail anyways because they tend
to be poorly-determined fitness values.

Let's then systematically summarize all MCMC chains.

In [ ]:
# Initialize dataframe to save fit values
df_fit = DF.DataFrame()

# Define percentiles to include
per = [2.5, 97.5, 16, 84]

# Loop through files
for f in chn_files
    ## EXTRACT METADATA ##
    # Split filename to extract metadata
    meta_str = split(split(f, "/")[end], "_")
    # Extract metadata
    env = replace(meta_str[2], "env" => "")
    rep = replace(meta_str[3], "rep" => "")
    rmT0 = parse(Bool, replace(meta_str[4], "rmT0.jld2" => ""))

    ## LOAD AND FORMAT CHAIN ##
    # Load chain and group object form .jld2 file
    chn, group = values(JLD2.load(f))

    # Search for variable names
    s_names = MCMCChains.namesingroup(chn, "s̲⁽ᵐ⁾")

    # Rename variables
    chn = MCMCChains.replacenames(
        chn, Dict(zip(s_names, ["s[$(x)]" for x in group]))
    )

    ## CONVERT TO TIDY DATAFRAME ##
    # Define variable fitness variable names
    var_names = MCMCChains.namesingroup(chn, "s")

    # Convert chain to tidy dataframe
    df = DF.stack(DF.DataFrame(chn[var_names]), DF.Not([:iteration, :chain]))

    ## FILTER 95% ##

    # Group data by barcode
    df_group = DF.groupby(df, :variable)

    # Initialize dataframe to save summaries
    df_filt = DF.DataFrame()

    # Loop through groups
    for g in df_group
        # Filter data
        DF.append!(
            df_filt,
            g[(g.value.≥StatsBase.percentile(g.value, 5)).&(g.value.≤StatsBase.percentile(g.value, 95)),
                :]
        )
    end # group

    # Compute summary statistics
    df_summary = DF.combine(
        DF.groupby(df_filt, :variable),
        :value => StatsBase.median,
        :value => StatsBase.mean,
        :value => StatsBase.std,
        :value => StatsBase.var,
        :value => StatsBase.skewness,
        :value => StatsBase.kurtosis,
    )

    # Loop through percentiles
    for p in per
        # Compute and add percentile
        DF.leftjoin!(
            df_summary,
            # Rename column from :value_function to :p_percentile
            DF.rename!(
                # Compute percentile p for each group
                DF.combine(
                    # Group MCMC chains by :variable, :value
                    DF.groupby(df_filt[:, [:variable, :value]], :variable),
                    # Define anonymous function to compute percentile
                    :value => x -> StatsBase.percentile(x, p)
                ),
                :value_function => Symbol("$(p)_percentile")
            );
            on=:variable
        )
    end # for


    # Add inference metadata
    DF.insertcols!(
        df_summary,
        :env .=> env,
        :rep .=> rep,
        :rmT0 .=> rmT0
    )

    # Append to dataframe
    DF.append!(df_fit, df_summary)
end # for

# Rename columns
DF.rename!(
    df_fit,
    :variable => :barcode,
    :value_median => :median,
    :value_mean => :mean,
    :value_std => :std,
    :value_var => :var,
    :value_skewness => :skewness,
    :value_kurtosis => :excess_kurtosis,
)

# Replace barcode name from "s[x]" to x
df_fit.barcode .= parse.(Int64, replace.(df_fit.barcode, "s[" => "", "]" => ""))

# Extract barcode metadata to be added
df_bc_meta = unique(data[:, [:barcode, :class, :gene, :ploidy, :type]])
# Add barcode metadata
DF.leftjoin!(df_fit, df_bc_meta; on=:barcode)

# Write dataframe to CSV file
CSV.write("$(git_root())/data/kinsler_2020/fitness_gaussian_fit.csv", df_fit)

first(df_fit, 10)

### Exploring summary statistics

Let's look at some of the correlation between a few of the variables.

For example, we expect the mean and the median to be almost mirror-images of
each other. Let's plot one against the other.

In [ ]:
# Intialize figure
fig = Figure(resolution=(300, 300))

# Add axis
ax = Axis(fig[1, 1], xlabel="fitness mean", ylabel="fitness median")

# Plot mean vs median
scatter!(ax, df_fit.mean, df_fit.median, markersize=5)

# Plot identity line
lines!(ax, [-4, 4], [-4, 4], color=:black)

# Seet axis limits
xlims!(ax, [-3, 2])
ylims!(ax, [-3, 2])

fig

These are as correlated as I expected.

In [ ]:
# Initialize figure
fig = Figure(resolution=(300 * 2, 300 * 2))

# Add axis
axes = [
    Axis(
        fig[i, j],
        ylabel="ECDF"
    ) for i = 1:2 for j = 1:2
]

# List quantities to plot
qs = ["mean", "var", "skewness", "excess_kurtosis"]

# Loop through quantities
for (i, q) in enumerate(qs)
    # Plot quantity
    ecdfplot!(axes[i], df_fit[:, q])
    # Add title
    axes[i].xlabel = "fitness $(q)"
end # for

fig

In [ ]:
# Intialize figure
fig = Figure(resolution=(300, 300))

# Add axis
ax = Axis(fig[1, 1], xlabel="fitness var", ylabel="fitness skewness")

# Plot mean vs median
scatter!(ax, df_fit.var, df_fit.skewness, markersize=5)

fig

### Replicate-to-replicate variability

Let's take a look at the replicate-to-replicate variability. First, let's plot 
the pairwise comparison

In [ ]:
# Group data by environment
df_group = DF.groupby(df_fit, :env)

# Find environments with two repeats
reps = [length(unique(g.rep)) for g in df_group] .≥ 2

# Keep groups with two repeats
df_group = df_group[reps]

# Define number of rows and columns
n_row, n_col = [4, 3]

# Initialize figure
fig = Figure(resolution=(350 * n_col, 350 * n_row))

# Add GridLayout
gl = fig[1, 1] = GridLayout()

# Add axis
axes = [
    Axis(
        gl[i, j],
        xlabel="replicate 1 median fitness",
        ylabel="replicate 2 median fitness",
    ) for i = 1:n_row for j = 1:n_col
]

# Loop through groups
for (i, data) in enumerate(df_group)
    # Unstack dataframe to plot columns
    d = DF.unstack(data[:, [:barcode, :rep, :median]], :rep, :median)

    # Plot Rep 1 vs Rep 2
    if first(data.env) == "M3"
        # Extract repeats to be used for M3
        m3 = [d.R6, d.R7]
        # Plot R1 vs R2
        scatter!(axes[i], m3..., markersize=5)
        # Plot identity line
        lines!(
            axes[i],
            repeat([[minimum(vcat(m3...)), maximum(vcat(m3...))]], 2)...;
            color=:black
        )
    else
        # Plot R1 vs R2
        scatter!(axes[i], d.R1, d.R2, markersize=5)
        # Plot identity line
        lines!(
            axes[i],
            repeat([[minimum(data.median), maximum(data.median)]], 2)...;
            color=:black
        )
    end # if

    # Add title
    axes[i].title = first(data.env)

end # for

fig

Let's now add errorbars to this plot. The error bars will represent the 68
percentile.

In [ ]:
# Group data by environment
df_group = DF.groupby(df_fit, :env)

# Find environments with two repeats
reps = [length(unique(g.rep)) for g in df_group] .≥ 2

# Keep groups with two repeats
df_group = df_group[reps]

# Define number of rows and columns
n_row, n_col = [4, 3]

# Initialize figure
fig = Figure(resolution=(350 * n_col, 350 * n_row))

# Add GridLayout
gl = fig[1, 1] = GridLayout()

# Add axis
axes = [
    Axis(
        gl[i, j],
        xlabel="replicate 1 fitness",
        ylabel="replicate 2 fitness",
    ) for i = 1:n_row for j = 1:n_col
]

# Loop through groups
for (i, data) in enumerate(df_group)
    # Unstack dataframe to plot columns (median)
    d = DF.sort(
        DF.unstack(data[:, [:barcode, :rep, :median]], :rep, :median),
        [:barcode]
    )

    # Unstack dataframe to plot columns (16 percentile)
    d_low = DF.sort(
        DF.unstack(
            data[:, [:barcode, :rep, Symbol("16.0_percentile")]],
            :rep,
            Symbol("16.0_percentile")
        ),
        [:barcode]
    )
    # Unstack dataframe to plot columns (84 percentile)
    d_high = DF.sort(
        DF.unstack(
            data[:, [:barcode, :rep, Symbol("84.0_percentile")]],
            :rep,
            Symbol("84.0_percentile")
        ),
        [:barcode]
    )


    # Plot Rep 1 vs Rep 2
    if first(data.env) == "M3"
        # Extract repeats to be used for M3
        m3 = [d.R6, d.R7]
        m3_low = [d_low.R6, d_low.R7]
        m3_high = [d_high.R6, d_high.R7]
        # Add y-axis error bars
        errorbars!(
            axes[i],
            m3[1],
            m3[2],
            abs.(m3[1] .- m3_low[1]),
            abs.(m3[1] .- m3_high[1]),
            color=(:gray, 0.5)
        )

        # Add x-axis error bars
        errorbars!(
            axes[i],
            m3[1],
            m3[2],
            abs.(m3[2] .- m3_low[2]),
            abs.(m3[2] .- m3_high[2]),
            color=(:gray, 0.5),
            direction=:x
        )
        # Plot R1 vs R2
        scatter!(axes[i], m3..., markersize=5)
        # Plot identity line
        lines!(
            axes[i],
            repeat([[minimum(vcat(m3...)), maximum(vcat(m3...))]], 2)...;
            color=:black
        )
    else
        # Add y-axis error bars
        errorbars!(
            axes[i],
            d.R1,
            d.R2,
            abs.(d.R1 .- d_low.R1),
            abs.(d.R1 .- d_high.R1),
            color=(:gray, 0.5)
        )

        # Add x-axis error bars
        errorbars!(
            axes[i],
            d.R1,
            d.R2,
            abs.(d.R2 .- d_low.R2),
            abs.(d.R2 .- d_high.R2),
            color=(:gray, 0.5),
            direction=:x
        )

        # Plot R1 vs R2
        scatter!(axes[i], d.R1, d.R2, markersize=5)

        # Plot identity line
        lines!(
            axes[i],
            repeat([[minimum(data.median), maximum(data.median)]], 2)...;
            color=:black
        )
    end # if

    # Add title
    axes[i].title = first(data.env)

end # for

fig

## Comparison with previous inference method

An important comparison to be made is with the previous inference method. Let's
load the previous fitness measurements.

In [ ]:
# Load previous-method fitness inferences
df_prev = CSV.read(
    "$(git_root())/data/kinsler_2020/tidy_fitness_mle.csv", DF.DataFrame
)

# Initialize array where to save rep number. NOTE: This is done to split the 
# `EC Batch #`
rep = ones(Int64, size(df_prev, 1))

# Extract unique `EC Batch #` environments
ec_env = unique(df_prev.environment[occursin.("EC Batch", df_prev.environment)])

# Loop through EC envs
for env in ec_env
    # Extract and save replicate number
    rep[df_prev.environment.==env] .= parse(Int64, split(env, " ")[end])
    # Modify environment to be `EC Batch` only
    df_prev[df_prev.environment.==env, :environment] .= "EC Batch"
end # for

# Add rep column
df_prev[!, :rep] = ["R$(r)" for r in rep]

first(df_prev[:, DF.Not(:additional_muts)], 3)

This dataframe uses a different naming convention for the environments. The file
`env_equivalence.csv` is a manually-curated mapping between (most) environment
names. Let's map these names to the new environments

In [ ]:
# Read environment naming convention
df_env = CSV.read(
    "$(git_root())/data/kinsler_2020/env_equivalence.csv", DF.DataFrame
)

# Drop environemnts that have a missing entry
DF.dropmissing!(df_env)

# Initialize array to save new environment name
# envs = Array{Union{String,Missing}}(missing, size(df_prev, 1))
envs = repeat(["None"], size(df_prev, 1))

# Loop through old environments
for row in eachrow(df_env)
    # Add corresponding environment
    envs[df_prev.environment.==row.old] .= row.new
end # for

# Add column to dataframe
df_prev[!, :env] = envs

first(df_prev[:, DF.Not(:additional_muts)], 3)

To plot the comparison, we must find the environment/replicate pairs that appear
in both datasets as well as the common barcodes.

In [ ]:
# Collect groups for previous inference
keys_prev = values.(keys(DF.groupby(df_prev, [:env, :rep])))

# Collect groups for new inference 
keys_fit = values.(keys(DF.groupby(df_fit, [:env, :rep])))

# Find intersecting groups
keys_intersect = intersect(keys_prev, keys_fit)

# Find intersecting barcodes
bc_intersect = intersect(unique(df_fit.barcode), unique(df_prev.barcode))

println("# envs = $(length(keys_intersect))")
println("# barcodes = $(length(bc_intersect)) out of $(length(unique(df_fit.barcode)))")


We see that a little over 100 barcodes were excluded from the original Kinsler
et al dataset. The authors did not explain why this is the case, so I'll need to
ask them directly.

Let's now plot the pairwise comparisons.

In [ ]:

# Define number of rows and columns in subplots
n_col, n_row = [4, 5]

# Initialize figure
fig = Figure(resolution=(350 * n_col, 350 * n_row))

# Add GridLayout
gl = fig[1, 1] = GridLayout()

# Add axis
axes = [
    Axis(
        gl[i, j],
        xlabel="Bayesian inference",
        ylabel="old method",
    ) for i = 1:n_row for j = 1:n_col
]

# Loop through groups
for (i, (env, rep)) in enumerate(keys_intersect)
    # Extract old and new inference
    data_bayes = DF.sort!(
        df_fit[
            (df_fit.env.==env).&(df_fit.rep.==rep).&([x ∈ bc_intersect for x in df_fit.barcode]),
            :],
        :barcode
    )
    data_prev = DF.sort!(
        df_prev[
            (df_prev.env.==env).&(df_prev.rep.==rep).&([x ∈ bc_intersect for x in df_prev.barcode]),
            :],
        :barcode
    )

    # Plot identity line
    lines!(
        axes[i],
        repeat([[
                minimum([data_bayes.median; data_prev.fitness]),
                maximum([data_bayes.median; data_prev.fitness])
            ]],
            2)...,
        color=:black
    )

    # Plot x-axis error-bar
    errorbars!(
        axes[i],
        data_bayes.median,
        data_prev.fitness,
        abs.(data_bayes.median .- data_bayes[:, Symbol("16.0_percentile")]),
        abs.(data_bayes.median .- data_bayes[:, Symbol("84.0_percentile")]),
        color=(:gray, 0.5),
        direction=:x
    )

    # Plot y-axis error-bar
    errorbars!(
        axes[i],
        data_bayes.median,
        data_prev.fitness,
        data_prev.error,
        color=(:gray, 0.5),
        direction=:y
    )

    # Plot centroid
    scatter!(axes[i], data_bayes.median, data_prev.fitness, markersize=7)

    # Set title
    axes[i].title = "$(env) | $(rep)"
end # for

fig

Although there are some suspiciously-diverging comparisons, these are mostly for
the M3 datasets, where I am less certain about the replicate number matching
with each other (something interesting to consider about the M3 data), the rest
of the comparisons look really good. As expected, the error bars from the
Bayesian inference look significantly bigger than with the older method.